In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>img[src*='#left'] { float: left; }</style>"))
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/USTC LP strategy
---

In this notebook we pull data from Binance and we run a static LP strategy.  
The pools are arbed at the beginning of each simulation step if it is profitable to do so.  
At the end of the simulation we compute and display various metrics for the strategy.    

![backtesting](https://letianzj.github.io/quanttrading-backtest/backtest_structure.png#left)  

Connection to Binance
---


The binance client allows to connect to Binance in order to pull historical data.  
In this example, credentials are saved as environment variables.

In [10]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

Creating trades
---

We use OHLVC data and create a buy order for a green candle or a sell order for a red one.  
If the pair is not listed, we create a composite one using data from each token (component) against a pivot, for example BUSD.  
In the example below we use data from LUNC/BUSD and USTC/BUSD to create trades for LUNC/USTC every hour.  
We then display, some metrics for our pair such as the price, the average price for the period or measures of how cointegrated the component prices are.

In [11]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure

base_pair, quote_pair = 'LUNC/BUSD', 'USTC/BUSD'
start, end = '2023-03-01 00:00:00', '2023-06-28 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

df_lunc = client.get_trade_data(base_pair, frequency, start, end)
df_ustc = client.get_trade_data(quote_pair, frequency, start, end)
df_trades = client.create_trade_data(base_pair, quote_pair, 0.001, frequency, start, end)
show(new_trade_figure(df_lunc, df_ustc, df_trades, "LUNC/USTC"))

Function 'plot_price_ratio' executed in 0.0194s
Function 'plot_scatterplot' executed in 0.0100s
Function 'new_trade_figure' executed in 0.0472s


Backtesting
---

We create a pair with an amount of liquidity in USD. We then run the backtesting between 01/03/2023 and 28/06/2023.  
Various metrics such as the P&L, price impact or impermanent loss are plotted.  
For the impermanent loss, the solid line shows the actual one from the simulation eg. the loss incured by people trading against the DEX. 
Whereas the dash line shows the theorical using this [formula](https://medium.com/auditless/how-to-calculate-impermanent-loss-full-derivation-803e8b2497b7).    

⚠️ **All units are expressed in quote currency unless explicitly specified. So in this example that's USTC**  

![backtesting](https://mtr-cdn.com/images/backtesting_strategies_on_forex.width-648.jpg#left)  

In [13]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.strategy import get_strategy

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote(base_pair, df_trades.price_1.iloc[0])
# USTC/BUSD market price
quote = MarketQuote(quote_pair, df_trades.price_2.iloc[0])
# create a 1,000,000 USD market for LUNC/USTC with 1% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.003)
# load strategy 
strategy = get_strategy("uni_v2")
# run simulation
simul = swap_simulation(mkt, df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul, plot_height=300))

Function 'trade_summary' executed in 0.0064s
Function 'sim_results' executed in 0.0193s
Function 'swap_simulation' executed in 0.1144s
Function 'new_pnl_figure' executed in 0.0155s
Function 'new_portfolio_figure' executed in 0.0278s
Function 'new_price_figure' executed in 0.0463s
Function 'new_fitted_pnl_figure' executed in 0.0369s
Function 'new_sim_price_impact_figure' executed in 0.0135s
Function 'new_roi_distrib_figure' executed in 0.0108s
Function 'new_pnl_arb_figure' executed in 0.0167s


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# display breakdown results
simul["breakdown"].head(20)

,side,arb_profit,price,price_impact,mid_price,mkt_price,spread,avg_price,current_base,current_quote,cp_invariant,total_fees_paid_quote,total_volume_base,total_volume_quote,asset_base_pct,hold_portfolio,current_portfolio,trade_pnl,total_pnl,roi,impermanent_loss,mkt_price_ratio,volume_base,volume_quote,fees_paid_quote,trade_pnl_pct,fees_pnl_pct,total_arb_profit,asset_quote_pct
trade_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-01 03:00:00,sell,0.00000,0.00607,0.00009,0.00607,0.00607,0.00000,0.00609,"3,033,162,632.34331","18,421,219.30083","55,874,554,025,485,464.00000",5.23165,"286,252.38455","-1,743.88410",0.50000,"36,842,443.99740","36,842,438.60166",-5.39574,-0.16408,-0.00000,0.00000,1.00000,"286,252.38455","-1,743.88410",5.23165,-0.00000,0.00000,0.00000,0.50000
2023-03-01 07:00:00,sell,0.00000,0.00607,0.00001,0.00607,0.00607,-0.00001,0.00609,"3,033,198,249.55970","18,421,002.99036","55,874,554,025,485,464.00000",5.88254,"321,869.60094","-1,960.84546",0.50000,"36,842,012.07070","36,842,005.98071",-6.08999,-0.20745,-0.00000,-0.00000,0.99876,"35,617.21639",-216.96136,0.65088,-0.00000,0.00000,0.00000,0.50000
2023-03-01 09:00:00,sell,111.65570,0.00604,0.00599,0.00600,0.00600,-0.00000,0.00606,"3,051,479,311.85509","18,310,644.87588","55,874,554,025,485,472.00000",337.95309,"18,602,931.89634","-112,651.03049",0.50000,"36,622,312.40638","36,621,289.75177","-1,022.65461",-684.70152,-0.00002,-0.00002,0.98781,"18,281,062.29540","-110,690.18503",332.07056,-0.00003,0.00001,111.65570,0.50000
2023-03-01 09:00:00,buy,0.00000,0.00600,-0.00002,0.00600,0.00600,-0.00000,0.00606,"3,051,420,406.36921","18,310,998.34977","55,874,554,025,485,472.00000",339.01670,"18,544,026.41046","-112,296.49299",0.50000,"36,623,013.98751","36,621,996.69954","-1,017.28797",-678.27127,-0.00002,-0.00002,0.98781,"-58,905.48588",354.53750,1.06361,-0.00003,0.00001,111.65570,0.50000
2023-03-01 10:00:00,buy,0.00000,0.00600,-0.00001,0.00600,0.00605,0.00005,0.00606,"3,051,397,192.83561","18,311,137.65087","55,874,554,025,485,472.00000",339.43586,"18,520,812.87685","-112,156.77273",0.50000,"36,623,290.47849","36,622,275.30174","-1,015.17675",-675.74088,-0.00002,-0.00000,0.99562,"-23,213.53360",139.72026,0.41916,-0.00003,0.00001,111.65570,0.50000
2023-03-01 14:00:00,buy,300.48218,0.00604,-0.00724,0.00609,0.00609,0.00000,0.00610,"3,029,468,337.90548","18,443,683.11310","55,874,554,025,485,480.00000",738.26875,"-3,408,042.05327","20,787.52238",0.50000,"36,887,327.17868","36,887,366.22619",39.04751,777.31626,0.00002,-0.00000,1.00229,"-21,928,854.93013","132,944.29511",398.83289,0.00000,0.00002,412.13788,0.50000
2023-03-01 14:00:00,buy,0.00000,0.00609,-0.00006,0.00609,0.00609,-0.00000,0.00610,"3,029,286,191.40018","18,444,792.10453","55,874,554,025,485,472.00000",741.60574,"-3,590,188.55857","21,899.85079",0.50000,"36,889,544.38646","36,889,584.20905",39.82259,781.42833,0.00002,-0.00000,1.00229,"-182,146.50530","1,112.32841",3.33699,0.00000,0.00002,412.13788,0.50000
2023-03-01 20:00:00,buy,0.00000,0.00609,-0.00009,0.00609,0.00612,0.00003,0.00610,"3,028,999,844.64721","18,446,535.78449","55,874,554,025,485,480.00000",746.85252,"-3,876,535.31154","23,648.77753",0.50000,"36,893,030.79771","36,893,071.56897",40.77126,787.62377,0.00002,-0.00001,1.00813,"-286,346.75297","1,748.92674",5.24678,0.00000,0.00002,412.13788,0.50000
2023-03-02 03:00:00,buy,0.00000,0.00609,-0.00003,0.00609,0.00607,-0.00002,0.00610,"3,028,923,297.59858","18,447,001.96594","55,874,554,025,485,480.00000",748.25527,"-3,953,082.36018","24,116.36174",0.50000,"36,893,962.96291","36,894,003.93188",40.96897,789.22424,0.00002,-0.00000,0.99966,"-76,547.04864",467.58421,1.40275,0.00000,0.00002,412.13788,0.50000
